In [1]:
import matplotlib.pyplot as plt
import numpy as np

import os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

2024-09-11 11:49:22.325867: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-11 11:49:22.326463: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-11 11:49:22.328748: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-11 11:49:22.334713: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-11 11:49:22.344201: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [2]:
from grav_lens import get_datasets
from grav_lens.models import test_model_image

In [3]:
from os.path import join
home_data = join("..", "data")
max_files = -1
batch_size = 256

train_dataset, val_dataset, test_dataset = get_datasets(data_index='1', max_files=max_files,batch_size=batch_size,
    home=home_data)

Using data folder: ../data/4


# Scaler Clasico

In [4]:
from utils import CustomScaler

# --- Ejemplo de uso del CustomScaler ---
scaler = CustomScaler()

# Ajustar el scaler a los datos de entrenamiento
scaler.fit_scaler(train_dataset)

# Normalizar un batch de datos
for X_batch, y_batch in train_dataset.take(1):
    y_batch_scaled = scaler.transform(y_batch)
    print("Scaled batch:", y_batch_scaled.shape)

# Desescalar para volver a los valores originales
y_batch_original = scaler.inverse_transform(y_batch_scaled)
print("Original batch:", y_batch_original.shape)


NameError: name 'StandardScaler' is not defined

In [ ]:

# Ajustar el scaler a los datos de entrenamiento
scaler.fit_scaler(train_dataset)
# Guardar el scaler ajustado
scaler.save_scaler('scaler.pkl')

# Cargar el scaler cuando lo necesites
scaler.load_scaler('scaler.pkl')

# Scaler Modificado

In [10]:
from sklearn.preprocessing import StandardScaler
import pickle
import numpy as np
import tensorflow as tf

class CustomScaler:
    def __init__(self):
        self.scaler = StandardScaler()

    def fit_scaler(self, dataset):
        """
        Función para ajustar el StandardScaler a los datos de entrenamiento.
        
        Parámetros:
        - dataset: Un iterable o generador de datos que contiene las imágenes en lotes (batch_size, height, width, 1)
        """
        y_data = []
        for X_batch, y_batch in dataset:
            y_batch = y_batch.numpy().reshape(-1)  # Aplanar el batch de imágenes
            y_data.extend(y_batch)

        y_data = np.array(y_data).reshape(-1, 1)
        self.scaler.fit(y_data)  # Ajustar el scaler a los datos

    def transform(self, y_batch):
        """
        Función para normalizar un batch de datos usando el scaler ajustado.
        
        Parámetros:
        - y_batch: Un batch de imágenes en forma de tensor (batch_size, height, width, 1)
        
        Retorna:
        - El batch normalizado.
        """
        y_batch = y_batch.numpy()  # Convertir a numpy
        y_batch_flat = y_batch.reshape(-1, 1)  # Aplanar
        y_batch_scaled = self.scaler.transform(y_batch_flat)  # Escalar
        y_batch_scaled = y_batch_scaled.reshape(y_batch.shape)  # Volver a la forma original
        return tf.convert_to_tensor(y_batch_scaled, dtype=tf.float32)  # Convertir de vuelta a tensor


    def inverse_transform(self, y_batch_scaled):
        """
        Función para desescalar un batch de datos usando el scaler ajustado.
        
        Parámetros:
        - y_batch_scaled: Un batch normalizado de imágenes (batch_size, height, width, 1)
        
        Retorna:
        - El batch desescalado a los valores originales.
        """
        y_batch_flat = y_batch_scaled.reshape(-1, 1)  # Aplanar
        y_batch_original = self.scaler.inverse_transform(y_batch_flat)  # Desescalar
        y_batch_original = y_batch_original.reshape(y_batch_scaled.shape)  # Volver a la forma original
        return y_batch_original

In [11]:
scaler = CustomScaler()
scaler.fit_scaler(train_dataset.take(5))

2024-09-11 11:41:03.632118: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [13]:
# Normalizar un batch de datos
for X_batch, y_batch in train_dataset.take(1):
    y_batch_scaled = scaler.transform(y_batch)
    print("Scaled batch:", y_batch_scaled)

Scaled batch: tf.Tensor(
[[[[-0.25536203]
   [-0.12367794]
   [ 0.01581572]
   ...
   [-0.4430014 ]
   [-0.5239892 ]
   [-0.46088183]]

  [[-0.36264464]
   [-0.2055072 ]
   [ 0.03516866]
   ...
   [-0.5231478 ]
   [-0.573213  ]
   [-0.4629854 ]]

  [[-0.32435948]
   [-0.1434516 ]
   [ 0.18481208]
   ...
   [-0.35296816]
   [-0.28481215]
   [-0.26840425]]

  ...

  [[ 0.665375  ]
   [ 0.8698429 ]
   [ 0.6939837 ]
   ...
   [-0.5412386 ]
   [-0.45120534]
   [-0.57994443]]

  [[ 0.5745003 ]
   [ 0.48572925]
   [ 0.17313722]
   ...
   [-0.9577475 ]
   [-0.84457487]
   [-0.91399306]]

  [[ 0.47731492]
   [ 0.2027977 ]
   [-0.12967314]
   ...
   [-1.0090748 ]
   [-1.038525  ]
   [-1.0511465 ]]]


 [[[ 1.1937943 ]
   [ 0.7486768 ]
   [ 0.49498498]
   ...
   [-0.13188191]
   [ 0.14842016]
   [ 0.58165246]]

  [[ 1.20305   ]
   [ 0.82650924]
   [ 0.36372158]
   ...
   [-0.3573857 ]
   [-0.00219618]
   [ 0.6102612 ]]

  [[ 0.9716562 ]
   [ 0.71375734]
   [ 0.21415703]
   ...
   [ 0.05717736]
   

In [ ]:
import pickle

# Guardar el IPCA en un archivo pickle
with open('standard_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

## Transformar con la FFT

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from sklearn.mixture import GaussianMixture

# Filtro FFT
def fft_filter_image(image, threshold_factor=1):
    """
    Aplica FFT 2D a una imagen, filtra los valores que están por encima del umbral basado en std.
    """
    fft_image = tf.signal.fft2d(tf.cast(image, tf.complex64))
    abs_fft_image = tf.abs(fft_image)
    
    # Calcular estadísticos de la FFT
    mean_value = tf.reduce_mean(abs_fft_image)
    std_value = tf.math.reduce_std(abs_fft_image)
    
    # Filtrar los valores por encima del threshold
    threshold = mean_value + threshold_factor * std_value
    mask = abs_fft_image > threshold
    
    # Aplicar la máscara y filtrar
    filtered_fft = tf.where(mask, fft_image, tf.zeros_like(fft_image))
    
    # Inversa de FFT para reconstruir la imagen filtrada
    filtered_image = tf.signal.ifft2d(filtered_fft)
    
    # Obtener parte real de la imagen filtrada
    return tf.math.real(filtered_image)

# Estadísticas de la imagen
def calculate_image_stats(image):
    mean_value = tf.reduce_mean(image)
    std_value = tf.math.reduce_std(image)
    return mean_value, std_value

# Thresholding
def apply_threshold(image, mean, std, threshold=2.0):
    """
    Aplica un threshold a la imagen para eliminar el ruido.
    """
    return tf.where(image > mean + threshold * std, image, tf.zeros_like(image))

# Ajuste Gaussian Mixture
def fit_gaussian_mixture(image, n_gaussians=5):
    """
    Ajusta un modelo Gaussian Mixture a la imagen.
    """
    # Obtener las coordenadas de los píxeles no cero
    x, y_coords = np.indices(image.shape)
    x = x.flatten()
    y_coords = y_coords.flatten()
    intensities = image.flatten()

    # Crear los datos para el GMM
    data = np.stack((x, y_coords, intensities), axis=1)
    
    # Ajustar un Gaussian Mixture Model (GMM)
    gmm = GaussianMixture(n_components=n_gaussians, covariance_type='full').fit(data)

    return gmm, gmm.means_, gmm.covariances_, gmm.weights_

# Pipeline para procesar la imagen
def process_image_pipeline(image, threshold_factor=1, n_gaussians=5, threshold_value=2.0):
    """
    Ejecuta el pipeline completo: filtro FFT, thresholding, y ajuste de Gaussian Mixture.
    """
    # Paso 1: Filtrar la imagen usando FFT
    filtered_image = fft_filter_image(image, threshold_factor)

    # Paso 2: Calcular estadísticas de la imagen filtrada
    mean_value, std_value = calculate_image_stats(filtered_image)

    # Paso 3: Aplicar un threshold para eliminar ruido
    image_thresholded = apply_threshold(filtered_image, mean_value, std_value, threshold=threshold_value)

    # Paso 4: Ajustar un modelo Gaussian Mixture Model (GMM) a la imagen filtrada y thresholded
    gmm, means, covariances, weights = fit_gaussian_mixture(image_thresholded.numpy(), n_gaussians)

    return {
        'filtered_image': filtered_image,
        'thresholded_image': image_thresholded,
        'gmm': gmm,
        'means': means,
        'covariances': covariances,
        'weights': weights
    }


In [ ]:
# Función para aplicar el pipeline a cada imagen en el dataset
def process_dataset_pipeline(dataset, threshold_factor=1, n_gaussians=5, threshold_value=2.0):
    processed_results = []
    
    for X_batch, y_batch in dataset:
        # Asegurarse de que y_batch tiene la forma (batch, 128, 128, 1)
        for i in range(y_batch.shape[0]):  # Iterar sobre el batch
            image = y_batch[i, :, :, 0]  # Extraer una imagen 2D (128, 128)
            
            # Ejecutar el pipeline de procesamiento
            result = process_image_pipeline(image, threshold_factor, n_gaussians, threshold_value)
            
            # Almacenar el resultado
            processed_results.append(result)
    
    return processed_results

# Aplicar la pipeline al dataset normalizado
processed_train = process_dataset_pipeline(train_dataset_scaled, threshold_factor=1, n_gaussians=5, threshold_value=2.0)

# Puedes hacer lo mismo para val_dataset_scaled y test_dataset_scaled


In [ ]:
# Visualizar la imagen original y filtrada de un ejemplo
import matplotlib.pyplot as plt

def visualize_results(result, original_image):
    plt.figure(figsize=(12, 5))

    # Imagen original
    plt.subplot(1, 3, 1)
    plt.imshow(original_image, cmap='viridis')
    plt.title("Imagen original")
    plt.colorbar()

    # Imagen filtrada
    plt.subplot(1, 3, 2)
    plt.imshow(result['filtered_image'].numpy(), cmap='viridis')
    plt.title("Imagen filtrada (FFT)")
    plt.colorbar()

    # Imagen con thresholding
    plt.subplot(1, 3, 3)
    plt.imshow(result['thresholded_image'].numpy(), cmap='viridis')
    plt.title("Imagen con threshold")
    plt.colorbar()

    plt.show()

# Ejemplo con un resultado del train dataset
example_result = processed_train[0]
original_image = y_batch[0, :, :, 0].numpy()  # Asegúrate de tener la imagen original
visualize_results(example_result, original_image)
